Transaction Analysis

In [16]:
# Transaction Analysis: 0xfaad385bd019c88f189a41fd8a60b95011be4df71b18ae853d6598112f2a39d8

import sys
from pathlib import Path
from web3 import Web3
import json

# Add project root to Python path
PROJECT_ROOT = Path(__file__).parent.parent if '__file__' in dir() else Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
    
from utils import (
    get_infura_web3,
    retry_with_backoff,
    get_token_decimals,
    TOKEN_COINGECKO_MAP
)


In [ ]:
# Enhanced Transaction Analysis - Identifying Unknown Event Hashes

import sys
from pathlib import Path
from web3 import Web3
import json
import requests

# Add project root to Python path
PROJECT_ROOT = Path(__file__).parent.parent if '__file__' in dir() else Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
    
from utils import (
    get_infura_web3,
    retry_with_backoff,
    get_token_decimals,
    TOKEN_COINGECKO_MAP,
    decode_liquidation_event,
    decode_flash_loan_event,
    FLASH_LOAN_EVENT_HASHES
)

# Connect to Ethereum
w3 = get_infura_web3(api_key="d031246997ba4c3abdde1f9cc049c42f")

tx_hash = "0xfaad385bd019c88f189a41fd8a60b95011be4df71b18ae853d6598112f2a39d8"

# Known Event Signatures
KNOWN_EVENTS = {
    "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef": "Transfer (ERC20)",
    "0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0": "Approval (ERC20)",
    "0xe413a321e8681d831f4dbccbca790d2952b56f977908e45be37335533e005286": "LiquidationCall (AAVE)",
    "0x804c9b842b2748a22bb64b345453a3de7ca54a6ca45ce00d415894979e22897a": "Borrow (AAVE)",
    "0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822": "Swap (Uniswap V2)",
    "0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67": "Swap (Uniswap V3)",
    "0x4c209b5fc8ad50758f13e2e1088ba56a560dff690a1c6fef26394f4c03821c4f": "Mint (Uniswap V2)",
    "0xcc16f5dbb4873280815c1ee09dbd06736cffcc184412cf7a71a0fdb75d397ca5": "Burn (Uniswap V2)",
    "0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad": "Sync (Uniswap V2)",
    "0x0559884fd3a460db3073b7fc896cc77986f16e378210ded43186175bf646fc5f": "AnswerUpdated (Chainlink)",
    "0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c": "Deposit (WETH)",
    "0x7fcf532c15f0a6db0bd6d0e038bea71d30d808c7d98cb3bf7268a95bf5081b65": "Withdrawal (WETH)",
}

# Add flash loan hashes
for proto_name, proto_hash in FLASH_LOAN_EVENT_HASHES.items():
    KNOWN_EVENTS[proto_hash.lower()] = f"FlashLoan ({proto_name.replace('_', ' ').title()})"

def decode_uint256(data_bytes, offset=0):
    """Decode uint256 from bytes"""
    if len(data_bytes) < offset + 32:
        return None
    return int.from_bytes(data_bytes[offset:offset+32], byteorder="big")

def decode_address(topic):
    """Extract address from topic"""
    if isinstance(topic, str):
        hex_str = topic[2:] if topic.startswith('0x') else topic
    elif hasattr(topic, 'hex'):
        hex_str = topic.hex()
    elif isinstance(topic, bytes):
        hex_str = topic.hex()
    else:
        hex_str = bytes(topic).hex()
    
    if len(hex_str) < 40:
        return None
    return w3.to_checksum_address("0x" + hex_str[-40:])

def analyze_unknown_event(log, event_hash):
    """Try to analyze an unknown event"""
    topics = log.get("topics", [])
    data = log.get("data", "0x")
    contract = log.get("address", "")
    
    if hasattr(data, 'hex'):
        data = "0x" + data.hex()
    elif isinstance(data, bytes):
        data = "0x" + data.hex()
    
    analysis = {
        "event_hash": event_hash,
        "contract": contract,
        "num_topics": len(topics),
        "data_length": len(data) - 2 if data.startswith("0x") else len(data),
        "topics": [],
        "data_fields": [],
        "possible_structure": None
    }
    
    # Extract addresses from topics
    for i, topic in enumerate(topics[1:], 1):  # Skip topic[0] which is event hash
        addr = decode_address(topic)
        if addr:
            analysis["topics"].append({
                "index": i,
                "type": "address",
                "value": addr
            })
        else:
            analysis["topics"].append({
                "index": i,
                "type": "unknown",
                "value": str(topic)[:20] + "..."
            })
    
    # Try to decode data fields
    if data != "0x" and len(data) > 2:
        data_bytes = bytes.fromhex(data[2:])
        num_fields = len(data_bytes) // 32
        
        for i in range(min(num_fields, 10)):  # Limit to 10 fields
            offset = i * 32
            value = decode_uint256(data_bytes, offset)
            if value is not None:
                analysis["data_fields"].append({
                    "index": i,
                    "value": value,
                    "value_hex": hex(value)
                })
    
    # Try to infer structure based on pattern
    if len(topics) == 1:
        analysis["possible_structure"] = "Anonymous event (no indexed parameters)"
    elif len(topics) == 2:
        analysis["possible_structure"] = "1 indexed parameter (likely address)"
    elif len(topics) == 3:
        analysis["possible_structure"] = "2 indexed parameters (likely 2 addresses)"
    elif len(topics) == 4:
        analysis["possible_structure"] = "3 indexed parameters (likely 3 addresses or address + uint256)"
    
    return analysis

def get_contract_name(address):
    """Try to get contract name from Etherscan"""
    try:
        url = f"https://api.etherscan.io/api"
        params = {
            "module": "contract",
            "action": "getsourcecode",
            "address": address,
            "apikey": "YourApiKeyToken"
        }
        response = requests.get(url, params=params, timeout=5)
        if response.status_code == 200:
            data = response.json()
            if data.get("result") and len(data["result"]) > 0:
                contract_name = data["result"][0].get("ContractName", "")
                if contract_name:
                    return contract_name
    except:
        pass
    return None

def find_log_by_index(receipt, log_index):
    """Find a log in receipt by its logIndex"""
    for log in receipt.get("logs", []):
        if log.get("logIndex") == log_index:
            return log
    return None

print("=" * 100)
print(f"COMPREHENSIVE EVENT ANALYSIS: {tx_hash}")
print("=" * 100)

# Get transaction and receipt
tx = retry_with_backoff(lambda: w3.eth.get_transaction(tx_hash))
receipt = retry_with_backoff(lambda: w3.eth.get_transaction_receipt(tx_hash))

tx_from = tx['from']
print(f"\n📋 Transaction Overview:")
print(f"  From: {tx_from}")
print(f"  To: {tx.get('to', 'Contract Creation')}")
print(f"  Block: {receipt['blockNumber']}")

# Collect all unique event hashes
all_event_hashes = {}
unknown_events = []

for log_index, log in enumerate(receipt['logs']):
    topics = log.get("topics", [])
    if not topics:
        continue
    
    topic0 = topics[0]
    if hasattr(topic0, 'hex'):
        event_hash = "0x" + topic0.hex()
    elif isinstance(topic0, bytes):
        event_hash = "0x" + topic0.hex()
    else:
        event_hash = topic0 if isinstance(topic0, str) else str(topic0)
    
    event_hash_lower = event_hash.lower()
    
    if event_hash_lower not in all_event_hashes:
        all_event_hashes[event_hash_lower] = {
            "hash": event_hash,
            "known": event_hash_lower in [k.lower() for k in KNOWN_EVENTS.keys()],
            "name": KNOWN_EVENTS.get(event_hash_lower, "Unknown"),
            "occurrences": [],
            "contracts": set()
        }
    
    all_event_hashes[event_hash_lower]["occurrences"].append({
        "log_index": log.get("logIndex", log_index),
        "contract": log.get("address", "")
    })
    all_event_hashes[event_hash_lower]["contracts"].add(log.get("address", ""))

# Identify unknown events
print(f"\n🔍 Event Hash Analysis:")
print("=" * 100)

for event_hash_lower, event_info in sorted(all_event_hashes.items()):
    if event_info["known"]:
        print(f"\n✅ {event_info['name']}")
        print(f"   Hash: {event_info['hash']}")
        print(f"   Occurrences: {len(event_info['occurrences'])}")
        print(f"   Contracts: {len(event_info['contracts'])}")
        for contract in list(event_info['contracts'])[:3]:
            print(f"     - {contract}")
    else:
        print(f"\n❓ UNKNOWN EVENT")
        print(f"   Hash: {event_info['hash']}")
        print(f"   Occurrences: {len(event_info['occurrences'])}")
        unknown_events.append(event_info)

# Detailed analysis of unknown events
if unknown_events:
    print(f"\n🔬 Detailed Analysis of Unknown Events:")
    print("=" * 100)
    
    for unknown in unknown_events:
        print(f"\n📌 Unknown Event Hash: {unknown['hash']}")
        print(f"   Occurrences: {len(unknown['occurrences'])}")
        
        # Analyze first occurrence - FIX: Find log by logIndex instead of using as array index
        first_occurrence = unknown['occurrences'][0]
        log = find_log_by_index(receipt, first_occurrence['log_index'])
        
        if log is None:
            print(f"   ⚠️  Could not find log with logIndex {first_occurrence['log_index']}")
            continue
            
        analysis = analyze_unknown_event(log, unknown['hash'])
        
        print(f"\n   Contract Analysis:")
        contract_addr = analysis['contract']
        print(f"     Address: {contract_addr}")
        
        # Try to get contract name
        contract_name = get_contract_name(contract_addr)
        if contract_name:
            print(f"     Name: {contract_name}")
        
        print(f"\n   Event Structure:")
        print(f"     Number of Topics: {analysis['num_topics']}")
        print(f"     Data Length: {analysis['data_length']} bytes ({analysis['data_length'] // 32} uint256 fields)")
        print(f"     Possible Structure: {analysis['possible_structure']}")
        
        if analysis['topics']:
            print(f"\n   Indexed Parameters (Topics):")
            for topic_info in analysis['topics']:
                print(f"     Topic {topic_info['index']}: {topic_info['type']}")
                if topic_info['type'] == 'address':
                    print(f"       Address: {topic_info['value']}")
        
        if analysis['data_fields']:
            print(f"\n   Data Fields (uint256 values):")
            for field in analysis['data_fields'][:5]:  # Show first 5
                print(f"     Field {field['index']}: {field['value']:,} ({field['value_hex']})")
                # Try to interpret as token amount
                if field['value'] > 0:
                    # Check if it could be a token amount (try common decimals)
                    for decimals in [6, 8, 18]:
                        human_readable = field['value'] / (10 ** decimals)
                        if 0.000001 < human_readable < 1000000000:
                            print(f"       → Could be {human_readable:,.6f} tokens (assuming {decimals} decimals)")

# Try to decode all events chronologically
print(f"\n🔄 Chronological Event Decoding:")
print("=" * 100)

events = []
flash_loan_hashes = [h.lower() for h in FLASH_LOAN_EVENT_HASHES.values()]

for log_index, log in enumerate(receipt['logs']):
    topics = log.get("topics", [])
    if not topics:
        continue
    
    topic0 = topics[0]
    if hasattr(topic0, 'hex'):
        event_hash = "0x" + topic0.hex()
    elif isinstance(topic0, bytes):
        event_hash = "0x" + topic0.hex()
    else:
        event_hash = topic0 if isinstance(topic0, str) else str(topic0)
    
    event_hash_lower = event_hash.lower()
    event_name = KNOWN_EVENTS.get(event_hash_lower, "Unknown")
    
    events.append({
        "log_index": log.get("logIndex", log_index),
        "event_hash": event_hash,
        "event_name": event_name,
        "contract": log.get("address", ""),
        "log": log
    })

events.sort(key=lambda x: x["log_index"])

step = 1
for event in events:
    print(f"\n📍 Step {step}: {event['event_name']}")
    print(f"   Contract: {event['contract']}")
    print(f"   Event Hash: {event['event_hash']}")
    
    if event['event_name'] == "Unknown":
        # Try to analyze it
        analysis = analyze_unknown_event(event['log'], event['event_hash'])
        print(f"   ⚠️  This is an unknown event - attempting analysis:")
        print(f"      Topics: {analysis['num_topics']}")
        print(f"      Data: {analysis['data_length']} bytes")
        if analysis['topics']:
            print(f"      Indexed params:")
            for topic_info in analysis['topics']:
                if topic_info['type'] == 'address':
                    print(f"        - Address: {topic_info['value']}")
        if analysis['data_fields']:
            print(f"      Data values:")
            for field in analysis['data_fields'][:3]:
                print(f"        - {field['value']:,}")
    
    step += 1

print(f"\n📊 Summary:")
print(f"  Total Events: {len(events)}")
known_count = sum(1 for e in events if e['event_name'] != "Unknown")
unknown_count = len(events) - known_count
print(f"  Known Events: {known_count}")
print(f"  Unknown Events: {unknown_count}")